In [77]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = 30
import os
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import datetime
import pymysql
import time
from datetime import date
import openpyxl
import glob
import shutil

# Selenium libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select



print("Importado con éxito")

Importado con éxito


In [26]:
try:
    connection = pymysql.connect(host="192.168.100.215",user="root", passwd="sqladmin", db="siogranos", port=3316)
    cursor = connection.cursor()
    base_de_datos = pd.read_sql_query (f"SELECT * FROM data", connection)
    connection.commit()
    connection.close()
    print("La base se ha cargado correctamente")
except:
    print("No se encontró la base de datos del SIO-GRANOS")

C:\Users\frolotti\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


La base se ha cargado correctamente


In [29]:
# Borramos los archivos del directorio en el que descargaremos los datos que faltan

directory = 'sio-granos/Archivos/*'
files = glob.glob(directory)

for file in files:
    if os.path.exists(file):
        os.remove(file)

In [30]:
old_max_day = 1
old_max_month = 6
old_max_year = 2022
if date.today().day==1:                  # El día corriente no está disponible para descargar, hay que
    today_day = 28                       # usar el día anterior
    if date.today().month==1:
        today_month = 11
        today_year = date.today().year-1
    else:
        today_month = date.today().month-2
else:    
    today_day = date.today().day-1
    today_month = date.today().month-1       # Enero lo cuentan como 0, y de ahí empiezan a sumar...
    today_year = date.today().year


In [62]:
base_de_datos['FECHA CONCERTACION'] = pd.to_datetime(base_de_datos['FECHA CONCERTACION'])
base_de_datos.sort_values(by=['FECHA CONCERTACION'], inplace = True)

base_de_datos.tail(10)

ParserError: Unknown string format: CÓRDOBA

In [70]:
# Fechas para la descarga de datos

old_max_day = int(base_de_datos['FECHA CONCERTACION'].iloc[-1][0:2])+1
old_max_month = int(base_de_datos['FECHA CONCERTACION'].iloc[-1][3:5])-1
old_max_year = int(base_de_datos['FECHA CONCERTACION'].iloc[-1][6:10])

print(old_max_day, old_max_month, old_max_year)


if date.today().day==1:                  # El día corriente no está disponible para descargar, hay que
    today_day = 28                       # usar el día anterior
    if date.today().month==1:
        today_month = 11
        today_year = date.today().year-1
    else:
        today_month = date.today().month-2
else:    
    today_day = date.today().day-1
    today_month = date.today().month-1      # Enero lo cuentan como 0, y de ahí empiezan a sumar...
    today_year = date.today().year



procedencia = 'CÓRDOBA'

print(today_day,today_month,today_year)

28 0 2023
15 1 2023


In [82]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.siogranos.com.ar/Consulta_publica/operaciones_informadas_exportar.aspx")
driver.maximize_window()
time.sleep(3)




# Fecha de Inicio:
fecha_inicio = driver.find_element(By.NAME,"txtFechaOperacionDesde").click()
time.sleep(1)
# Mes:
select_mes_inicio = Select(driver.find_element(By.CLASS_NAME,"ui-datepicker-month"))
select_mes_inicio.select_by_value(str(old_max_month))
time.sleep(1)
# Año:
select_anio_inicio = Select(driver.find_element(By.CLASS_NAME,"ui-datepicker-year"))
select_anio_inicio.select_by_value(str(old_max_year))
time.sleep(1)
# Dia:
day_inicio = driver.find_element(By.XPATH,"//a[@class='ui-state-default' and text()="+str(old_max_day)+"]")
day_inicio.click()


# Fecha de hoy:
fecha_hoy = driver.find_element(By.NAME,"txtFechaOperacionHasta").click()
time.sleep(1)
# Mes:
select_mes = Select(driver.find_element(By.CLASS_NAME,"ui-datepicker-month"))
select_mes.select_by_value(str(today_month))
time.sleep(1)
# Año:
select_anio = Select(driver.find_element(By.CLASS_NAME,"ui-datepicker-year"))
select_anio.select_by_value(str(today_year))
time.sleep(1)
# Dia:
day = driver.find_element(By.XPATH,"//a[@class='ui-state-default' and text()="+str(today_day)+"]")
day.click()

# Procedencia:
sel = Select(driver.find_element(By.ID, "ddlProvincia"))
sel.select_by_visible_text(procedencia)



csv = driver.find_element(By.ID,"btn_generar_csv")
csv.click()

#fecha_hasta = driver.find_element(By.NAME,"txtFechaOperacionHasta").click()




In [83]:
folder_path = r'C:\Users\frolotti\Downloads'
file_type = r'\*csv'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)

print(max_file)

C:\Users\frolotti\Downloads\operaciones_informadas (2).csv


In [84]:
shutil.move(max_file, r'C:\Users\frolotti\Documents\PythonProjects\siogranos\SIO-GRANOS\sio-granos\Archivos')

'C:\\Users\\frolotti\\Documents\\PythonProjects\\siogranos\\SIO-GRANOS\\sio-granos\\Archivos\\operaciones_informadas (2).csv'

In [107]:
# Este código para agregar nuevos datos a la base SQL, no usarlo si no hay datos a agregar.
# Ingresar a: https://www.siogranos.com.ar/Consulta_publica/operaciones_informadas_exportar.aspx
# Descargar los días a agregar con filtro de Cordoba como procedencia Provincia y guardarlos en la carpeta de SIO-GRANOS.

# Creo un Dataframe vacio y una lista con los archivos que se encuentran en el directorio, en caso de querer 
# agregar un período de tiempo muy largo:
data = pd.DataFrame()
fichero = os.listdir('sio-granos/Archivos/')

#Creo un "for" que me lea todos los archivos y me vaya apendizando las bases a nuestro Dataframe "data":

for base in fichero:
    if (base!='.ipynb_checkpoints' and base!='SIO-GRANOS.ipynb' and base!='Dolar.xlsx' and base!='Archivos'):    # Compila todos los archivos de la carpeta excepto estos dos.
        apendice = pd.read_excel(f"sio-granos/Archivos/{base}")
        data = pd.concat([data,apendice])


data

,FECHA OPERACION,FECHA CONCERTACION,OPERACION,TIPO,PRECIO,PRODUCTO,CANT. (TN),CALIDAD,CALIDAD ADICIONAL,PROCEDENCIA PCIA,PROCEDENCIA LOCALID.,PRECIO/TN MONEDA,PRECIO/TN MONTO,LUGAR ENTREGA,FECHA ENTR. DESDE,FECHA ENTR. HASTA,CONDICION PAGO,ES FINAL,COSECHA,NRO INSTANCIA OPERACION
0,15/02/2023 09:05:36 p.m.,15/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,GIRASOL,1000.00,Cámara,NaN,CÓRDOBA,VILLA HUIDOBRO,U$S,400.0,Zona 25/En destino,15/02/2023 12:00:00 a.m.,17/03/2023 12:00:00 a.m.,Contra entrega,SI,COSECHA 22/23,1
1,15/02/2023 09:05:32 p.m.,15/02/2023 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,GIRASOL,350.00,Cámara,NaN,CÓRDOBA,TOSQUITA,U$S,400.0,Zona 25/En destino,15/02/2023 12:00:00 a.m.,17/03/2023 12:00:00 a.m.,Contra entrega,SI,COSECHA 22/23,1
2,15/02/2023 08:46:14 p.m.,30/06/2022 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,SOJA,3.97,Cámara,NaN,CÓRDOBA,GENERAL LEVALLE,$,47940.0,Zona 9/En destino,30/06/2022 12:00:00 a.m.,23/01/2023 12:00:00 a.m.,Contra entrega,SI,COSECHA 21/22,1
3,15/02/2023 08:45:29 p.m.,29/06/2022 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,SOJA,5.00,Cámara,NaN,CÓRDOBA,OLIVA,$,47940.0,Zona 11/En destino,29/06/2022 12:00:00 a.m.,23/01/2023 12:00:00 a.m.,Contra entrega,SI,COSECHA 21/22,1
4,15/02/2023 08:44:42 p.m.,19/04/2022 12:00:00 a.m.,Contrato,Compraventa,Precio Hecho,SOJA,3.97,Cámara,NaN,CÓRDOBA,OLIVA,$,50475.0,Zona 11/En destino,19/04/2022 12:00:00 a.m.,23/01/2023 12:00:00 a.m.,Contra entrega,SI,COSECHA 21/22,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13814,28/01/2023 12:01:04 a.m.,20/10/2022 12:00:00 a.m.,Contrato,Compraventa,Fijar Precio/Prec. Cam.,MAIZ,30.30,Cámara,NaN,CÓRDOBA,ITALO,0,0.0,B.Blanca/En destino,20/10/2022 12:00:00 a.m.,30/10/2022 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1
13815,28/01/2023 12:00:40 a.m.,19/10/2022 12:00:00 a.m.,Contrato,Compraventa,Fijar Precio/Prec. Cam.,MAIZ,518.61,Cámara,NaN,CÓRDOBA,LABOULAYE,0,0.0,B.Blanca/En destino,19/10/2022 12:00:00 a.m.,19/11/2022 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1
13816,28/01/2023 12:00:25 a.m.,18/10/2022 12:00:00 a.m.,Contrato,Compraventa,Fijar Precio/Prec. Cam.,MAIZ,30.22,Cámara,NaN,CÓRDOBA,ITALO,0,0.0,B.Blanca/En destino,18/10/2022 12:00:00 a.m.,25/10/2022 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1
13817,28/01/2023 12:00:21 a.m.,18/10/2022 12:00:00 a.m.,Contrato,Compraventa,Fijar Precio/Prec. Cam.,MAIZ,28.99,Cámara,NaN,CÓRDOBA,INRIVILLE,0,0.0,Rosario S/En destino,18/10/2022 12:00:00 a.m.,20/10/2022 12:00:00 a.m.,Contra entrega,NO,COSECHA 21/22,1


In [108]:
baseSQL = pd.DataFrame()

for i in data.columns:
    baseSQL[i] = data[i].astype(str)


# Busco los nombres de columnas y paso el tipo de dato del dataframe a uno compatible con SQL:    
    
def getColumnDtypes(dataTypes):
    dataList = []
    for x in dataTypes:
        if(x == 'int64'):
            dataList.append('int')
        elif (x == 'float64'):
            dataList.append('float')
        elif (x == 'bool'):
            dataList.append('boolean')
        else:
            dataList.append('varchar(50)')
    return dataList

columnas_nombres = list(baseSQL.columns.values)
columnas_tipo_de_datos = getColumnDtypes(baseSQL.dtypes)




In [109]:


# Defino el statement para crear la tabla:
crear_tabla_statement = 'CREATE TABLE IF NOT EXISTS data ('
for i in range(len(columnas_tipo_de_datos)):
    crear_tabla_statement = crear_tabla_statement +"\n`" + columnas_nombres[i] +  "` " + columnas_tipo_de_datos[i] + ', '
crear_tabla_statement = crear_tabla_statement[:-2] + ');'
    
    # Me conecto a la base

connection = pymysql.connect(host="192.168.100.215",user="root", passwd="sqladmin", db="siogranos", port=3316)
cursor = connection.cursor()

    # Creo la tabla:
try: 
    cursor.execute(crear_tabla_statement)
except:
    print("La tabla ya existe, se va a proceder a cargar los datos:")


    # Preparo la inserción de datos:
cols = "`,`".join([str(i) for i in baseSQL.columns.tolist()])

for i,row in baseSQL.iterrows():
    sql = "INSERT INTO data (`" + cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))


#cursor.execute("UPDATE data SET `Fecha` = STR_TO_DATE(`FECHA OPERACION`,'%Y-%m-%d') where Fecha IS NULL;")

connection.commit()
connection.close()
print("Carga exitosa.")


Carga exitosa.
